In [5]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

# 예시 좌표 - 10개의 지역 (lat, lon)
locations = np.array([
    [37.561, 126.982], [37.562, 126.975], [37.565, 126.984], [37.570, 126.975],
    [37.577, 126.986], [37.580, 126.978], [37.585, 126.990], [37.590, 126.980],
    [37.595, 126.976], [37.599, 126.985]
])

# 원하는 클러스터 개수 선택 (2, 3, 4)
cluster_count_options = [5, 4, 3]  # 각 클러스터당 2, 3, 4개의 군집 생성

# 최소 거리 군집화 함수
def optimal_clustering(locations, cluster_counts):
    best_clusters = {}
    for n_clusters in cluster_counts:
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        labels = kmeans.fit_predict(locations)
        
        # 클러스터 내 거리의 총합을 계산
        intra_distances = np.sum([cdist(locations[labels == i], [kmeans.cluster_centers_[i]], 'euclidean').sum()
                                  for i in range(n_clusters)])
        
        best_clusters[n_clusters] = (labels, intra_distances)
    
    # 가장 작은 intra_distances를 가진 클러스터 선택
    optimal_clusters = min(best_clusters.items(), key=lambda x: x[1][1])
    return optimal_clusters[1][0]  # 최적화된 클러스터의 라벨 반환

# 최적 클러스터 구하기
best_labels = optimal_clustering(locations, cluster_count_options)

# 클러스터링 결과 출력
for i, cluster in enumerate(set(best_labels)):
    cluster_locations = locations[best_labels == cluster]
    print(f"Cluster {i+1} locations: {cluster_locations}")


Cluster 1 locations: [[ 37.58  126.978]]
Cluster 2 locations: [[ 37.59  126.98 ]
 [ 37.595 126.976]]
Cluster 3 locations: [[ 37.561 126.982]
 [ 37.562 126.975]
 [ 37.565 126.984]
 [ 37.57  126.975]]
Cluster 4 locations: [[ 37.577 126.986]
 [ 37.585 126.99 ]]
Cluster 5 locations: [[ 37.599 126.985]]


In [6]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

# 예시 좌표 - 10개의 지역 (lat, lon)
locations = np.array([
    [37.561, 126.982], [37.562, 126.975], [37.565, 126.984], [37.570, 126.975],
    [37.577, 126.986], [37.580, 126.978], [37.585, 126.990], [37.590, 126.980],
    [37.595, 126.976], [37.599, 126.985]
])

# 원하는 클러스터 개수 선택 (2, 3, 4)
cluster_count_options = [5, 4, 3]  # 각 클러스터당 2, 3, 4개의 군집 생성

# 최소 거리 군집화 함수
def optimal_clustering(locations, cluster_counts):
    best_clusters = {}
    for n_clusters in cluster_counts:
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        labels = kmeans.fit_predict(locations)
        
        # 클러스터 내 거리의 총합을 계산
        intra_distances = np.sum([cdist(locations[labels == i], [kmeans.cluster_centers_[i]], 'euclidean').sum()
                                  for i in range(n_clusters)])
        
        best_clusters[n_clusters] = (labels, intra_distances)
    
    # 가장 작은 intra_distances를 가진 클러스터 선택
    optimal_clusters = min(best_clusters.items(), key=lambda x: x[1][1])
    labels = optimal_clusters[1][0]

    # 1개짜리 클러스터가 있으면 병합
    unique_labels, counts = np.unique(labels, return_counts=True)
    for label, count in zip(unique_labels, counts):
        if count == 1:
            single_point_idx = np.where(labels == label)[0][0]  # 1개인 클러스터의 인덱스 찾기
            
            # 가장 가까운 클러스터로 병합
            distances = cdist([locations[single_point_idx]], locations[labels != label], 'euclidean')
            closest_label = labels[labels != label][np.argmin(distances)]
            labels[single_point_idx] = closest_label  # 병합 수행
            
    return labels

# 최적 클러스터 구하기
best_labels = optimal_clustering(locations, cluster_count_options)

# 클러스터링 결과 출력
for i, cluster in enumerate(set(best_labels)):
    cluster_locations = locations[best_labels == cluster]
    print(f"Cluster {i+1} locations: {cluster_locations}")


Cluster 1 locations: [[ 37.59  126.98 ]
 [ 37.595 126.976]
 [ 37.599 126.985]]
Cluster 2 locations: [[ 37.561 126.982]
 [ 37.562 126.975]
 [ 37.565 126.984]
 [ 37.57  126.975]]
Cluster 3 locations: [[ 37.577 126.986]
 [ 37.58  126.978]
 [ 37.585 126.99 ]]
